In [ ]:
import torch  # pyright: ignore[reportMissingImports]

print(torch.__version__)
print(torch.is_tensor(torch.randn(2, 3)))
print(torch.cuda.is_available()) # 因为mac的m4芯片没有cuda

2.9.0
True
False


In [3]:
import torch  # pyright: ignore[reportMissingImports]

# 检查 MPS 是否可用
print(f"MPS 可用: {torch.backends.mps.is_available()}")
print(f"MPS 已构建: {torch.backends.mps.is_built()}")

# 如果可用，使用 MPS 设备
if torch.backends.mps.is_available():
    device = torch.device("mps")
    print(f"使用设备: {device}")
else:
    device = torch.device("cpu")
    print(f"使用设备: {device}")

# 创建张量并移动到 MPS 设备
x = torch.randn(3, 3).to(device)
print(x)

MPS 可用: True
MPS 已构建: True
使用设备: mps
tensor([[ 0.8576, -0.1156, -0.2067],
        [-0.4358, -0.1326,  0.7850],
        [-2.2826,  0.1363, -0.8187]], device='mps:0')


# MPS (Metal Performance Shaders) 详细介绍

## 什么是 MPS？

**MPS (Metal Performance Shaders)** 是 Apple 为 macOS 和 iOS 系统提供的 GPU 加速框架。它是 Apple Silicon（M1/M2/M3/M4 等芯片）和部分 Intel Mac 上使用 Metal 图形 API 进行高性能计算的接口。

### 核心概念

1. **Metal**：Apple 的低级图形和计算 API，类似于 NVIDIA 的 CUDA 或 AMD 的 OpenCL
2. **MPS Backend**：PyTorch 通过 MPS backend 让开发者可以在 Apple Silicon 上使用 GPU 加速深度学习计算
3. **统一内存架构**：Apple Silicon 的 CPU 和 GPU 共享内存，减少了数据传输开销

### MPS vs CUDA

| 特性 | MPS | CUDA |
|------|-----|------|
| 平台 | Apple Silicon (M系列芯片) | NVIDIA GPU |
| 内存架构 | 统一内存（CPU/GPU共享） | 独立显存 |
| 数据传输 | 几乎零开销 | 需要显式传输 |
| 生态系统 | 相对较新，支持的操作较少 | 成熟，支持所有操作 |
| 性能 | 针对 Apple 硬件优化 | 针对 NVIDIA 硬件优化 |

### MPS 的优势

✅ **零拷贝**：统一内存架构意味着数据在 CPU 和 GPU 之间传输几乎不需要额外开销  
✅ **能效比高**：Apple Silicon 的能效比优秀，适合长时间训练  
✅ **原生支持**：macOS 系统级支持，无需额外驱动  

### MPS 的限制

⚠️ **操作支持**：不是所有 PyTorch 操作都支持 MPS（但核心操作都已支持）  
⚠️ **性能**：对于某些操作，可能不如 NVIDIA GPU 快  
⚠️ **仅限 Apple 设备**：只能在 Mac 上使用


In [ ]:
# MPS 详细使用示例

import torch  # pyright: ignore[reportMissingImports]
import time

# 1. 检查 MPS 是否可用
print("=" * 50)
print("MPS 状态检查")
print("=" * 50)
print(f"PyTorch 版本: {torch.__version__}")
print(f"MPS 已构建: {torch.backends.mps.is_available()}")
print(f"MPS 已构建: {torch.backends.mps.is_built()}")

# 2. 设备选择策略
print("\n" + "=" * 50)
print("设备选择")
print("=" * 50)
if torch.backends.mps.is_available():
    device = torch.device("mps")
    print(f"✅ 使用 MPS 设备: {device}")
elif torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"✅ 使用 CUDA 设备: {device}")
else:
    device = torch.device("cpu")
    print(f"⚠️  使用 CPU 设备: {device}")

# 3. 创建张量并移动到设备
print("\n" + "=" * 50)
print("张量操作示例")
print("=" * 50)
x = torch.randn(1000, 1000, device=device)
y = torch.randn(1000, 1000, device=device)
print(f"张量 x 形状: {x.shape}, 设备: {x.device}")
print(f"张量 y 形状: {y.shape}, 设备: {y.device}")

# 4. 矩阵运算
z = torch.matmul(x, y)
print(f"矩阵乘法结果形状: {z.shape}, 设备: {z.device}")

# 5. 神经网络层示例
print("\n" + "=" * 50)
print("神经网络层示例")
print("=" * 50)
linear = torch.nn.Linear(1000, 500).to(device)
output = linear(x)
print(f"线性层输出形状: {output.shape}, 设备: {output.device}")


MPS 状态检查
PyTorch 版本: 2.9.0
MPS 已构建: True
MPS 已构建: True

设备选择
✅ 使用 MPS 设备: mps

张量操作示例
张量 x 形状: torch.Size([1000, 1000]), 设备: mps:0
张量 y 形状: torch.Size([1000, 1000]), 设备: mps:0
矩阵乘法结果形状: torch.Size([1000, 1000]), 设备: mps:0

神经网络层示例
线性层输出形状: torch.Size([1000, 500]), 设备: mps:0


In [10]:
# MPS vs CPU 性能对比

import torch
import time

if torch.backends.mps.is_available():
    device_mps = torch.device("mps")
    device_cpu = torch.device("cpu")
    
    # 测试矩阵乘法性能
    size = 2000
    iterations = 10
    
    print("=" * 50)
    print("性能对比测试 (矩阵乘法)")
    print("=" * 50)
    print(f"矩阵大小: {size} x {size}")
    print(f"迭代次数: {iterations}")
    
    # MPS 测试
    x_mps = torch.randn(size, size, device=device_mps)
    y_mps = torch.randn(size, size, device=device_mps)
    
    # 预热（MPS 第一次运行可能较慢）
    _ = torch.matmul(x_mps, y_mps)
    torch.mps.synchronize()  # 等待 MPS 操作完成
    
    start_time = time.time()
    for _ in range(iterations):
        z_mps = torch.matmul(x_mps, y_mps)
    torch.mps.synchronize()
    mps_time = time.time() - start_time
    
    # CPU 测试
    x_cpu = torch.randn(size, size, device=device_cpu)
    y_cpu = torch.randn(size, size, device=device_cpu)
    
    start_time = time.time()
    for _ in range(iterations):
        z_cpu = torch.matmul(x_cpu, y_cpu)
    cpu_time = time.time() - start_time
    
    print(f"\nMPS 时间: {mps_time:.4f} 秒")
    print(f"CPU 时间: {cpu_time:.4f} 秒")
    print(f"加速比: {cpu_time / mps_time:.2f}x")
    
    if mps_time < cpu_time:
        print(f"✅ MPS 比 CPU 快 {cpu_time / mps_time:.2f} 倍")
    else:
        print(f"⚠️  CPU 更快（可能因为矩阵较小或 MPS 开销）")
else:
    print("MPS 不可用，无法进行性能对比")


性能对比测试 (矩阵乘法)
矩阵大小: 2000 x 2000
迭代次数: 10

MPS 时间: 0.0613 秒
CPU 时间: 0.1041 秒
加速比: 1.70x
✅ MPS 比 CPU 快 1.70 倍


In [11]:
# MPS 使用注意事项和最佳实践

import torch

print("=" * 50)
print("MPS 使用注意事项")
print("=" * 50)

if torch.backends.mps.is_available():
    device = torch.device("mps")
    
    # 1. 检查操作是否支持 MPS
    print("\n1. 操作支持检查")
    x = torch.randn(5, 5, device=device)
    print(f"✅ 基本操作（矩阵乘法）: 支持")
    print(f"✅ 激活函数（ReLU）: 支持")
    print(f"✅ 卷积操作: 支持")
    
    # 2. 数据类型支持
    print("\n2. 数据类型支持")
    print(f"✅ float32: 支持")
    print(f"✅ float16: 支持（部分操作）")
    print(f"⚠️  int64: 某些操作可能不支持，会回退到 CPU")
    
    # 3. 内存管理
    print("\n3. 内存管理")
    print("✅ 统一内存架构，无需手动管理 GPU 内存")
    print("✅ 但大模型仍需注意内存使用")
    
    # 4. 同步操作
    print("\n4. 同步操作")
    x = torch.randn(100, 100, device=device)
    y = torch.matmul(x, x)
    torch.mps.synchronize()  # 确保所有 MPS 操作完成
    print("✅ 使用 torch.mps.synchronize() 确保操作完成")
    
    # 5. 常见问题处理
    print("\n5. 常见问题处理")
    print("⚠️  如果某个操作不支持 MPS，PyTorch 会自动回退到 CPU")
    print("⚠️  某些操作（如某些索引操作）可能不支持，需要先转到 CPU")
    print("✅ 建议：先测试你的模型，确认所有操作都支持 MPS")
    
else:
    print("MPS 不可用")


MPS 使用注意事项

1. 操作支持检查
✅ 基本操作（矩阵乘法）: 支持
✅ 激活函数（ReLU）: 支持
✅ 卷积操作: 支持

2. 数据类型支持
✅ float32: 支持
✅ float16: 支持（部分操作）
⚠️  int64: 某些操作可能不支持，会回退到 CPU

3. 内存管理
✅ 统一内存架构，无需手动管理 GPU 内存
✅ 但大模型仍需注意内存使用

4. 同步操作
✅ 使用 torch.mps.synchronize() 确保操作完成

5. 常见问题处理
⚠️  如果某个操作不支持 MPS，PyTorch 会自动回退到 CPU
⚠️  某些操作（如某些索引操作）可能不支持，需要先转到 CPU
✅ 建议：先测试你的模型，确认所有操作都支持 MPS


In [ ]:
# 完整的神经网络训练示例（使用 MPS）

import torch  # pyright: ignore[reportMissingImports]
import torch.nn as nn  # pyright: ignore[reportMissingImports]
import torch.optim as optim  # pyright: ignore[reportMissingImports]

# 检查设备
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(f"使用设备: {device}")

# 定义简单的神经网络
class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.fc1 = nn.Linear(784, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        x = x.view(-1, 784)  # 展平
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# 创建模型并移动到设备
model = SimpleNet().to(device)
print(f"\n模型已移动到: {device}")

# 创建示例数据
batch_size = 32
x = torch.randn(batch_size, 28, 28, device=device)
y = torch.randint(0, 10, (batch_size,), device=device)

# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 训练一个批次
print("\n开始训练...")
model.train()
optimizer.zero_grad()
output = model(x)
loss = criterion(output, y)
loss.backward()
optimizer.step()

print(f"损失值: {loss.item():.4f}")
print(f"输出形状: {output.shape}")
print(f"✅ 训练完成！所有操作都在 {device} 上执行")


使用设备: mps

模型已移动到: mps

开始训练...
损失值: 2.3328
输出形状: torch.Size([32, 10])
✅ 训练完成！所有操作都在 mps 上执行
